In [1]:
import random
import numpy as np
class StochasticNetwork:
    def __init__(self, num_nodes, arcs, arc_lengths):
        self.num_nodes = num_nodes
        self.arcs = arcs
        self.arc_lengths = arc_lengths


    def get_random_length(self, arc):
        dist = self.arc_lengths[arc]
        if dist[0] == 'N':
            mean, stddev = dist[1], dist[2]
            return np.random.normal(mean, stddev)
        elif dist[0] == 'U':
            low, high = dist[1], dist[2]
            return np.random.uniform(low, high)
        elif dist[0] == 'EXP':
            rate = dist[1]
            return np.random.exponential(rate)
        elif dist[0] == 'T':
            low, mode, high = dist[1], dist[2], dist[3]
            return np.random.triangular(low, mode, high)


    def simulate_path_length(self, path, num_simulations=5000):
        lengths = []
        for _ in range(num_simulations):
            length = sum(self.get_random_length(arc) for arc in path)
            lengths.append(length)
        return np.mean(lengths), lengths


    def calculate_probability(self, path, threshold, num_simulations=5000):
        if not path:
            return 0.0
        _, lengths = self.simulate_path_length(path, num_simulations)
        count = sum(1 for l in lengths if l <= threshold)
        prob = count / num_simulations
        return min(max(prob, 0.0), 1.0)


    def calculate_alpha_shortest(self, path, alpha, num_simulations=5000):
        _, lengths = self.simulate_path_length(path, num_simulations)
        sorted_lengths = sorted(lengths)
        index = int((1 - alpha) * num_simulations)
        return sorted_lengths[index]



In [3]:
class GeneticAlgorithm:
    def __init__(self, network, pop_size=30, crossover_prob=0.2, mutation_prob=0.2, generations=800,a=0.05):
        self.network = network
        self.pop_size = pop_size
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob
        self.generations = generations
        self.population = []
        self.a=a


    def initialize_population(self, start, end):
        for _ in range(self.pop_size):
            path = self.random_path(start, end)
            self.population.append(path)


    def random_path(self, start, end):
        current = start
        path = []
        visited = set()
        while current!= end:
            visited.add(current)
            next_nodes = [arc[1] for arc in self.network.arcs if arc[0] == current]
            if not next_nodes:
                break
            next_node = random.choice(next_nodes)
            path.append((current, next_node))
            current = next_node
        return path


    def rank_based_evaluation(self):
        sorted_population = sorted(self.population, key=lambda path: self.fitness(path), reverse=True)
        a = self.a
        rank_evaluation = []
        for i in range(len(sorted_population)):
            rank_score = a * (1 - a) ** i
            rank_evaluation.append((sorted_population[i], rank_score))
        return rank_evaluation


    def selection(self):
        ranked_population = self.rank_based_evaluation()
        total_rank_score = sum(rank_score for _, rank_score in ranked_population)
        probabilities = [rank_score / total_rank_score for _, rank_score in ranked_population]
        selected_index = np.random.choice(len(ranked_population), p=probabilities)
        return ranked_population[selected_index][0]


    def crossover(self, chromosome1, chromosome2):
        common_nodes = set(chromosome1).intersection(set(chromosome2))
        if common_nodes:
            common_node = random.choice(list(common_nodes))
            index1 = chromosome1.index(common_node)
            index2 = chromosome2.index(common_node)
            new_chromosome1 = chromosome1[:index1 + 1] + chromosome2[index2 + 1:]
            new_chromosome2 = chromosome2[:index2 + 1] + chromosome1[index1 + 1:]
            return new_chromosome1, new_chromosome2
        else:
            return chromosome1, chromosome2


    def mutate(self, path):
        if random.random() < self.mutation_prob:
            index = random.randint(0, len(path) - 1)
            start_node = path[index][0]
            mutated_path = path[:index]
            mutated_path += self.random_path(start_node, path[-1][1])
            return mutated_path
        return path


    def run(self, start, end, objective='mean', threshold=None, alpha=None):
        self.initialize_population(start, end)
        for generation in range(self.generations):
            new_population = []
            for _ in range(self.pop_size // 2):
                parent1 = self.selection()
                parent2 = self.selection()
                if random.random() < self.crossover_prob:
                    child1, child2 = self.crossover(parent1, parent2)
                else:
                    child1, child2 = parent1, parent2
                new_population.append(self.mutate(child1))
                new_population.append(self.mutate(child2))
            self.population = new_population
        # 确保传递所需参数
        best_path = max(self.population, key=lambda path: self.fitness(path, objective, threshold, alpha))
        return best_path


    def fitness(self, path, objective='mean', threshold=None, alpha=None):
        if objective == 'mean':
            mean_length, _ = self.network.simulate_path_length(path)
            return 1 / mean_length
        elif objective == 'probability' and threshold is not None:
            prob = self.network.calculate_probability(path, threshold)
            return prob
        elif objective == 'alpha' and alpha is not None:
            alpha_length = self.network.calculate_alpha_shortest(path, alpha)
            return 1 / alpha_length



In [4]:
nodes = list(range(1, 24))
arcs = [(1, 2), (1, 3), (1, 4), (1, 5), (2, 6), (2, 7), (3, 8), (4, 7), (4, 11), (5, 8), (5, 11), (5, 12), (6, 9), (6, 10),
        (7, 10), (7, 11), (8, 12), (8, 13), (9, 16), (10, 16), (10, 17), (11, 14), (11, 17), (12, 14), (12, 15), (13, 15),
        (13, 19), (14, 21), (15, 18), (15, 19), (16, 20), (17, 20), (17, 21), (18, 21), (18, 22), (18, 23), (19, 22),
        (20, 23), (21, 23), (22, 23)]

# 定义弧长分布函数
arc_distributions = {
    (1, 2): ('N', 12, 1),
    (1, 3): ('N', 9, 1),
    (1, 4): ('T', 8, 10, 12),
    (1, 5): ('N', 7, 1),
    (2, 6): ('T', 5, 7, 15),
    (2, 7): ('U', 6, 11),
    (3, 8): ('U', 10, 16),
    (4, 7): ('EXP', 20),
    (4, 11): ('T', 6, 10, 13),
    (5, 11): ('U', 7, 13),
    (5, 12): ('EXP', 13),
    (5, 8): ('T', 6, 9, 10),
    (6, 9): ('T', 6, 8, 10),
    (6, 10): ('T', 10, 11, 14),
    (7, 10): ('U', 9, 12),
    (7, 11): ('U', 6, 8),
    (8, 12): ('U', 5, 9),
    (8, 13): ('N', 5, 1),
    (9, 16): ('EXP', 7),
    (10, 16): ('U', 12, 16),
    (10, 17): ('T', 15, 17, 19),
    (11, 17): ('EXP', 9),
    (11, 14): ('N', 9, 1),
    (12, 14): ('T', 10, 13, 15),
    (12, 15): ('N', 12, 2),
    (13, 15): ('T', 10, 12, 14),
    (13, 19): ('T', 17, 18, 19),
    (14, 21): ('N', 11, 1),
    (15, 18): ('T', 8, 9, 11),
    (15, 19): ('N', 7, 1),
    (16, 20): ('T', 9, 10, 12),
    (17, 20): ('T', 7, 11, 12),
    (17, 21): ('U', 6, 8),
    (18, 21): ('N', 15, 2),
    (18, 23): ('T', 5, 7, 9),
    (18, 22): ('EXP', 5),
    (19, 22): ('U', 15, 17),
    (20, 23): ('T', 13, 14, 15),
    (21, 23): ('T', 12, 13, 15),
    (22, 23): ('U', 4, 6)
}
network = StochasticNetwork(nodes, arcs, arc_distributions)
ga = GeneticAlgorithm(network)

In [62]:
# 测试期望路径长度最小化
best_path_mean = ga.run(1, 23, objective='mean')
mean_length, _ = network.simulate_path_length(best_path_mean)
print(f"最优路径 (期望路径长度最小化): {best_path_mean}")
print(f"该路径的期望长度: {mean_length:.2f}")
# 测试概率路径优化
threshold = 70  # 设置路径长度阈值
best_path_prob = ga.run(1, 23, objective='probability', threshold=threshold)
probability = network.calculate_probability(best_path_prob, threshold)
print(f"最优路径 (概率路径优化，阈值={threshold}): {best_path_prob}")
print(f"该路径小于阈值 {threshold} 的概率: {probability:.2%}")
# 测试 α-最短路径
alpha = 0.9  
best_path_alpha = ga.run(1, 23, objective='alpha', alpha=alpha)
alpha_length = network.calculate_alpha_shortest(best_path_alpha, alpha)
print(f"最优路径 (α-最短路径，置信度={alpha}): {best_path_alpha}")
print(f"该路径在置信度 {alpha} 下的最短长度: {alpha_length:.2f}")


最优路径 (期望路径长度最小化): [(1, 5), (5, 11), (11, 17), (17, 21), (21, 23)]
该路径的期望长度: 46.26
最优路径 (概率路径优化，阈值=20): [(1, 5), (5, 11), (11, 17), (17, 21), (21, 23)]
该路径小于阈值 20 的概率: 0.00%
最优路径 (α-最短路径，置信度=0.9): [(1, 5), (5, 11), (11, 17), (17, 21), (21, 23)]
该路径在置信度 0.9 下的最短长度: 37.63


In [7]:
alpha = 0.9  
best_path_alpha = ga.run(1, 23, objective='alpha', alpha=alpha)
alpha_length = network.calculate_alpha_shortest(best_path_alpha, alpha)
print(f"最优路径 (α-最短路径，置信度={alpha}): {best_path_alpha}")
print(f"该路径在置信度 {alpha} 下的最短长度: {alpha_length:.2f}")

最优路径 (α-最短路径，置信度=0.9): [(1, 5), (5, 11), (11, 17), (17, 21), (21, 23)]
该路径在置信度 0.9 下的最短长度: 37.54


In [13]:
threshold = 70  # 设置路径长度阈值
best_path_prob = ga.run(1, 23, objective='probability', threshold=threshold)
probability = network.calculate_probability(best_path_prob, threshold)
print(f"最优路径 (概率路径优化，阈值={threshold}): {best_path_prob}")
print(f"该路径小于阈值 {threshold} 的概率: {probability:.2%}")

最优路径 (概率路径优化，阈值=70): [(1, 5), (5, 11), (11, 14), (14, 21), (21, 23)]
该路径小于阈值 70 的概率: 100.00%


In [9]:
threshold = 80  # 设置路径长度阈值
best_path_prob = ga.run(1, 23, objective='probability', threshold=threshold)
probability = network.calculate_probability(best_path_prob, threshold)
print(f"最优路径 (概率路径优化，阈值={threshold}): {best_path_prob}")
print(f"该路径小于阈值 {threshold} 的概率: {probability:.2%}")

最优路径 (概率路径优化，阈值=80): [(1, 5), (5, 11), (11, 14), (14, 21), (21, 23)]
该路径小于阈值 80 的概率: 100.00%


In [11]:
threshold = 50  # 设置路径长度阈值
best_path_prob = ga.run(1, 23, objective='probability', threshold=threshold)
probability = network.calculate_probability(best_path_prob, threshold)
print(f"最优路径 (概率路径优化，阈值={threshold}): {best_path_prob}")
print(f"该路径小于阈值 {threshold} 的概率: {probability:.2%}")

最优路径 (概率路径优化，阈值=50): [(1, 5), (5, 11), (11, 17), (17, 21), (21, 23)]
该路径小于阈值 50 的概率: 75.42%


In [58]:
# 创建网络
arcs = [(1, 2), (1, 3), (2, 4), (3, 4), (4, 5)]
arc_lengths = {
    (1, 2): ('N', 10, 2),
    (1, 3): ('U', 5, 15),
    (2, 4): ('EXP', 10),
    (3, 4): ('T', 3, 4, 5),
    (4, 5): ('N', 7, 1),
}

network = StochasticNetwork(5, arcs, arc_lengths)
ga = GeneticAlgorithm(network, pop_size=20, generations=50)

# 期望路径长度最小化
best_path_mean = ga.run(1, 5, objective='mean')
mean_length, _ = network.simulate_path_length(best_path_mean)
print(f"最优路径 (期望路径长度最小化): {best_path_mean}")
print(f"该路径的期望长度: {mean_length:.2f}")

# 概率路径优化
threshold = 20
best_path_prob = ga.run(1, 5, objective='probability', threshold=threshold)
probability = network.calculate_probability(best_path_prob, threshold)
print(f"最优路径 (概率路径优化): {best_path_prob}")
print(f"该路径小于阈值 {threshold} 的概率: {probability:.2%}")

# α-最短路径
alpha = 0.9
best_path_alpha = ga.run(1, 5, objective='alpha', alpha=alpha)
alpha_length = network.calculate_alpha_shortest(best_path_alpha, alpha)
print(f"最优路径 (α-最短路径): {best_path_alpha}")
print(f"该路径在置信度 {alpha} 下的最短长度: {alpha_length:.2f}")


最优路径 (期望路径长度最小化): [(1, 3), (3, 4), (4, 5)]
该路径的期望长度: 21.01
最优路径 (概率路径优化): [(1, 3), (3, 4), (4, 5)]
该路径小于阈值 20 的概率: 38.84%
最优路径 (α-最短路径): [(1, 3), (3, 4), (4, 5)]
该路径在置信度 0.9 下的最短长度: 17.02


In [13]:
# 定义网络
arcs = [(1, 2), (2, 3), (3, 4)]
arc_lengths = {
    (1, 2): ('N', 10, 2),
    (2, 3): ('U', 5, 15),
    (3, 4): ('N', 8, 1)
}

network = StochasticNetwork(num_nodes=4, arcs=arcs, arc_lengths=arc_lengths)
ga = GeneticAlgorithm(network, pop_size=20, generations=500, crossover_prob=0.6, mutation_prob=0.3)

# 概率路径优化测试
threshold = 25
best_prob_path = ga.run(1, 4, objective='probability', threshold=threshold)
probability = network.calculate_probability(best_prob_path, threshold)

print(f"最优路径 (概率路径优化，阈值={threshold}): {best_prob_path}")
print(f"该路径小于阈值 {threshold} 的概率: {probability:.2%}")


最优路径 (概率路径优化，阈值=25): [(1, 2), (2, 3), (3, 4)]
该路径小于阈值 25 的概率: 21.70%


In [ ]:
# 定义网络
arcs = [(1, 2), (2, 5), (1, 3), (3, 5), (1, 4), (4, 5)]
arc_lengths = {
    (1, 2): ('N', 12, 3),
    (2, 5): ('U', 8, 12),
    (1, 3): ('N', 10, 1),
    (3, 5): ('T', 6, 9, 12),
    (1, 4): ('N', 15, 2),
    (4, 5): ('U', 5, 10)
}

network = StochasticNetwork(num_nodes=5, arcs=arcs, arc_lengths=arc_lengths)
ga = GeneticAlgorithm(network, pop_size=50, generations=1000, crossover_prob=0.7, mutation_prob=0.4)

# 概率路径优化测试
threshold = 20
best_prob_path = ga.run(1, 5, objective='probability', threshold=threshold)
probability = network.calculate_probability(best_prob_path, threshold)

print(f"最优路径 (概率路径优化，阈值={threshold}): {best_prob_path}")
print(f"该路径小于阈值 {threshold} 的概率: {probability:.2%}")


In [ ]:
# 采样三条弧的分布
num_simulations=10000
arc1 = np.random.normal(10, 1, num_simulations)  # 正态分布
arc2 = np.random.triangular(6, 9,12, num_simulations)  

# 计算总长度
total_lengths = arc1 + arc2 

# 小于阈值的比例
probability = np.mean(total_lengths <= 20)

print(f"Probability (T ≤ 20): {probability:.4f}")


In [ ]:
# 定义网络
arcs = [
    (1, 2), (2, 6), (6, 10), 
    (1, 3), (3, 7), (7, 10), 
    (1, 4), (4, 8), (8, 10), 
    (1, 5), (5, 9), (9, 10)
]
arc_lengths = {
    (1, 2): ('N', 10, 2),
    (2, 6): ('U', 8, 12),
    (6, 10): ('N', 15, 3),
    (1, 3): ('N', 9, 2),
    (3, 7): ('T', 6, 8, 10),
    (7, 10): ('N', 14, 2),
    (1, 4): ('N', 12, 3),
    (4, 8): ('U', 7, 11),
    (8, 10): ('N', 10, 1),
    (1, 5): ('N', 14, 3),
    (5, 9): ('T', 5, 7, 10),
    (9, 10): ('N', 13, 2)
}

network = StochasticNetwork(num_nodes=10, arcs=arcs, arc_lengths=arc_lengths)
ga = GeneticAlgorithm(network, pop_size=100, generations=1500, crossover_prob=0.7, mutation_prob=0.4)

# 概率路径优化测试
threshold = 35
best_prob_path = ga.run(1, 10, objective='probability', threshold=threshold)
probability = network.calculate_probability(best_prob_path, threshold)

print(f"最优路径 (概率路径优化，阈值={threshold}): {best_prob_path}")
print(f"该路径小于阈值 {threshold} 的概率: {probability:.2%}")
